In [ ]:
!pip install SimpleITK
!pip install skan 
!pip install tifffile
!pip install plotly==5.3.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:
import SimpleITK as sitk
from skimage.morphology import skeletonize, thin, medial_axis, skeletonize_3d
from scipy import ndimage
from skan import skeleton_to_csgraph, Skeleton
from skan import summarize
from skan import draw
import tifffile as tiff

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import glob
from tqdm.notebook import tnrange

In [19]:
def load_img_from_tiff(path2img):
    """
    Parameters
    ----------
    path2img: str 
        path to image Tiff file
    
    Returns
    -------
    img_array: np.array
        image data in numpy format
    """
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

In [20]:
single_thresh = load_img_from_tiff("/content/drive/MyDrive/mydata/sea_urchin_data/Test n.1")
# single_thresh = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)

single_thresh = single_thresh.astype('float64')
single_thresh = (single_thresh - np.min(single_thresh))/np.ptp(single_thresh)
skimage_skeleton, dist = medial_axis(single_thresh, return_distance=True)
skan_skeleton = Skeleton(dist*skimage_skeleton)

lee94_skimage_skeleton = skeletonize(single_thresh, method='lee')
lee94_skimage_skeleton=lee94_skimage_skeleton/255
lee_skan_skeleton=Skeleton(dist*lee94_skimage_skeleton)
df=summarize(lee_skan_skeleton)

In [21]:
#rename columns:
df = df.rename(columns={'image-coord-src-0': 'src-y', 
                        'image-coord-src-1': 'src-x',
                        'image-coord-dst-0': 'dst-y',
                        'image-coord-dst-1': 'dst-x',
                        'mean-pixel-value': 'thickness'})

#drop columns
df.drop([#'mean-pixel-value',
         'stdev-pixel-value',
         'coord-src-0',
         'coord-src-1',
         'coord-dst-0',
         'coord-dst-1'], axis=1, inplace=True)

In [22]:
def get_slope(x1, y1, x2, y2):
    if x1==x2:
        return ""
    m = (y2-y1)/(x2-x1)
    return m

#initiate blank columns
df["slope"] = 0
df["tortuosity"] = 0

#compute and assign values of slope and tortuosity
for index, row in df.iterrows():
    df.loc[index, 'slope']=get_slope(row["src-x"], row["src-y"], row["dst-x"], row["dst-y"])
    df.loc[index, 'tortuosity']=row["branch-distance"]/row["euclidean-distance"]

df.to_csv("single_summary.csv")

In [23]:
branches = pd.DataFrame(df['thickness'])
branches['source_node_id']=df['node-id-src']
branches['destination_node_id']=df['node-id-dst']
branches.to_csv("branches_lee.csv", index=False)
branches.tail()

,thickness,source_node_id,destination_node_id
726,3.552071,7704,7757
727,2.118389,7704,7783
728,2.404508,7711,7881
729,2.033724,7757,7880
730,2.342973,7773,7800


In [24]:
nodes_1 = pd.DataFrame(df["node-id-src"])
nodes_2=pd.DataFrame(df["node-id-dst"])

nodes_1 = nodes_1.rename(columns={'node-id-src': 'node_id'})
nodes_2 = nodes_2.rename(columns={'node-id-dst': 'node_id'})

nodes_1["node_coordinate_x"]=df['src-x']
nodes_1["node_coordinate_y"]=df['src-y']

nodes_2["node_coordinate_x"]=df['dst-x']
nodes_2["node_coordinate_y"]=df['dst-y']

final_nodes=nodes_1.append(nodes_2)

final_nodes=final_nodes.drop_duplicates(subset=['node_id',], keep='last').reset_index()
final_nodes = final_nodes.drop(['index'], axis=1)
final_nodes

final_nodes.to_csv("nodes_lee.csv", index=False)

In [25]:
# generate_node_image (includes dst nodes)
xs_=np.array(final_nodes['node_coordinate_x'])
ys_=np.array(final_nodes['node_coordinate_y'])
node_image = np.zeros([305,305], dtype=np.uint8)
# print(node_image.shape)
node_image[ys_.astype(np.uint16), xs_.astype(np.uint16)] = 4.

In [26]:
#compute porosity
def compute_porosity_2d(img):
    number_of_white_pix = np.sum(img == 1)  
    number_of_black_pix = np.sum(img == 0)
    porosity=number_of_black_pix/(number_of_white_pix+number_of_black_pix)
    return(porosity)

compute_porosity_2d(single_thresh)

0.5397688793335125

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
f, axarr = plt.subplots(2,2)

axarr[0,0].imshow(single_thresh+lee94_skimage_skeleton, cmap='jet')
axarr[0,0].set_title("Skeleton")
axarr[0,1].imshow(node_image+lee94_skimage_skeleton+single_thresh, cmap='inferno')
axarr[0,1].set_title("Branch Nodes")
axarr[1,0].imshow(dist)
axarr[1,0].set_title("Distance from nearest void")
axarr[1,1].imshow(dist*lee94_skimage_skeleton+single_thresh)
axarr[1,1].set_title("Distance from nearest void (skeleton)")

In [28]:
#branch path coords and thicknesses -> csv
branch_path_coords=[]

branch_thics=skan_skeleton.path_means()
# refer https://jni.github.io/skan/api/skan.csr.html

for i in range(len(branch_thics)):
    path_arr=skan_skeleton.path_coordinates(i).tolist()
    # print(path_arr.tolist())
    #replace newlines**** w commas
    branch_path_coords.append(path_arr)
    # break

branch_path_coords_and_thickness=pd.DataFrame(branch_thics, columns=['thickness'])
branch_path_coords_and_thickness['path_coordinates']=branch_path_coords
# branch_path_coords_and_thickness['thickness']=branch_thics

branch_path_coords_and_thickness
branch_path_coords_and_thickness.to_csv("branch_path_coords_and_thickness.csv", index=False)

In [29]:
#Thickness, src_x, src_y, dst_x, dst_y -> csv

thickness_and_coordinates = pd.DataFrame(df['thickness'])
thickness_and_coordinates['start_x']=df['src-x']
thickness_and_coordinates['start_y']=df['src-y']
thickness_and_coordinates['end_x']=df['dst-x']
thickness_and_coordinates['end_y']=df['dst-y']

thickness_and_coordinates.tail()
thickness_and_coordinates.to_csv("thickness_and_endpoint_coordinates.csv", index=False )